In [1]:
import pandas as pd
import numpy as np

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',') #dtype={1:'str'}

population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,07/30/2020,07/21/2020,10/29/2020,VFC,Alameda,Alameda County,7.0,Closure Permanent
1,09/16/2020,04/27/2020,10/29/2020,Araca Merchandise LP.,San Fernando,Los Angeles County,89.0,Layoff Permanent
2,10/23/2020,09/25/2020,10/29/2020,"STAR Sports Theatre Arts & Recreation, Inc.",Culver City,Los Angeles County,60.0,Layoff Permanent
3,09/16/2020,04/25/2020,10/29/2020,Panasonic Avionics Corporation,Lake Forest,Orange County,582.0,Layoff Permanent
4,09/03/2020,05/08/2020,10/29/2020,Molson Coors Beverage Company,Baldwin Park,Los Angeles County,436.0,Closure Permanent


In [4]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
9466,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name,86.0,Layoff Permanent
9467,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name,49.0,Closure Permanent
9468,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name,11.0,Closure Permanent
9469,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name,18.0,Layoff Unknown at this time
9470,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No County Name,50.0,Closure Permanent


## California 2020 Layoffs

We are selecting 2020 records by isolating the year on the `effective date` column.
By choosing `effective date` we make sure that these layoffs actually occurred in 2020, as some layoff notices can be issues much earlier the layoff date.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Effective Date'].str[-4:]
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_2020 = cali_layoffs[cali_layoffs['Year'] == '2020']
cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')
cali_2020.tail()

<ipython-input-5-957d264fb98a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5689,10/30/2019,01/03/2020,11/01/2019,"Ashford University, LLC",San Diego,San Diego County,218,Layoff Permanent,2020
5691,11/01/2019,01/10/2020,11/01/2019,American General Life Insurance Company,Pasadena,Los Angeles County,40,Closure Permanent,2020
5708,10/24/2019,01/01/2020,10/28/2019,"Hotel Del Coronado, Curio Collection By Hilton...",Coronado,San Diego County,160,Layoff Temporary,2020
5711,10/17/2019,05/24/2020,10/23/2019,Wells Fargo,Concord,Contra Costa County,357,Layoff Permanent,2020
5753,09/26/2019,01/16/2020,09/30/2019,"Forever 21 Logistics, LLC",Los Angeles,Los Angeles County,1170,Closure Permanent,2020


In [6]:
cali_2020['County'] = cali_2020['County'].str.strip()
cali_2020.loc[~(cali_2020['County'].str.endswith('County')),'County'] = cali_2020['County'] + " County"

<ipython-input-6-0b43f9e4a6a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['County'] = cali_2020['County'].str.strip()


In [7]:
cali_population.head(5)

,California,"39,512,223"
0,".Alameda County, California",1671329
1,".Alpine County, California",1129
2,".Amador County, California",39752
3,".Butte County, California",219186
4,".Calaveras County, California",45905


In [8]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [9]:
len(cali_2020)

5658

In [10]:
cali_merge = pd.merge(cali_2020, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,07/30/2020,07/21/2020,10/29/2020,VFC,Alameda,Alameda County,7,Closure Permanent,2020,1671329,Alameda County
1,09/16/2020,04/27/2020,10/29/2020,Araca Merchandise LP.,San Fernando,Los Angeles County,89,Layoff Permanent,2020,10039107,Los Angeles County
2,10/23/2020,09/25/2020,10/29/2020,"STAR Sports Theatre Arts & Recreation, Inc.",Culver City,Los Angeles County,60,Layoff Permanent,2020,10039107,Los Angeles County
3,09/16/2020,04/25/2020,10/29/2020,Panasonic Avionics Corporation,Lake Forest,Orange County,582,Layoff Permanent,2020,3175692,Orange County
4,09/03/2020,05/08/2020,10/29/2020,Molson Coors Beverage Company,Baldwin Park,Los Angeles County,436,Closure Permanent,2020,10039107,Los Angeles County


In [11]:
layoffs_by_county = cali_merge.groupby(['County Names','Year']).agg('sum')
layoffs = layoffs_by_county.merge(cali_population, how="left", on="County Names")
layoffs.drop(columns=['Counties', 'Population_x'], inplace=True)
layoffs.rename(columns={"Population_y":"Population"}, inplace=True)
layoffs['Percentage Laid Off'] = (layoffs['Employees']/layoffs['Population']) * 100
layoffs.sort_values(by='Percentage Laid Off', ascending=False)

,County Names,Employees,Population,Percentage Laid Off
18,Mariposa County,2262,17203,13.148869
33,San Francisco County,56652,881549,6.426415
21,Mono County,879,14444,6.085572
12,Inyo County,539,18039,2.987971
32,San Diego County,75458,3338330,2.260352
36,San Mateo County,16987,766573,2.215966
23,Napa County,2975,137744,2.159804
25,Orange County,64764,3175692,2.039367
0,Alameda County,32756,1671329,1.959877
15,Los Angeles County,191307,10039107,1.905618


In [12]:
cali_2019 = cali_layoffs[cali_layoffs['Year'] == '2019']
cali_2019['Employees'] = cali_2019['Employees'].astype('Int64')
cali_2019['County'] = cali_2019['County'].str.strip()
cali_2019.loc[~(cali_2019['County'].str.endswith('County')),'County'] = cali_2019['County'] + " County"
cali_2019

<ipython-input-12-d2d3be101e82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2019['Employees'] = cali_2019['Employees'].astype('Int64')
<ipython-input-12-d2d3be101e82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2019['County'] = cali_2019['County'].str.strip()
/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5478,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Fullerton,Orange County,204,Closure Permanent,2019
5492,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Buena Park,Orange County,45,Layoff Permanent,2019
5607,12/20/2019,12/20/2019,12/30/2019,"Event Solutions International, Inc. dba MotusO...",Long Beach,Los Angeles County,38,Layoff Permanent,2019
5608,12/30/2019,12/31/2019,12/30/2019,Taylor Lane Organic Coffee,Petaluma,Sonoma County,6,Closure Permanent,2019
5609,10/16/2019,11/01/2019,12/27/2019,Barneys New York,San Francisco,San Francisco County,100,Closure Permanent,2019
...,...,...,...,...,...,...,...,...,...
6705,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",Ontario,San Bernardino County,16,Closure Permanent,2019
6706,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",San Diego,San Diego County,84,Closure Permanent,2019
6707,06/28/2019,08/30/2019,06/28/2019,"Coloredge, Inc.",Burbank,Los Angeles County,32,Closure Permanent,2019
6708,06/28/2019,08/31/2019,06/28/2019,Verity Business Services,San Jose,Santa Clara County,5,Layoff Permanent,2019


In [13]:
cali_merge_2019 = pd.merge(cali_2019, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge_2019.drop(columns='Counties', inplace=True)
cali_merge_2019.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Fullerton,Orange County,204,Closure Permanent,2019,3175692,Orange County
1,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Buena Park,Orange County,45,Layoff Permanent,2019,3175692,Orange County
2,12/20/2019,12/20/2019,12/30/2019,"Event Solutions International, Inc. dba MotusO...",Long Beach,Los Angeles County,38,Layoff Permanent,2019,10039107,Los Angeles County
3,12/30/2019,12/31/2019,12/30/2019,Taylor Lane Organic Coffee,Petaluma,Sonoma County,6,Closure Permanent,2019,494336,Sonoma County
4,10/16/2019,11/01/2019,12/27/2019,Barneys New York,San Francisco,San Francisco County,100,Closure Permanent,2019,881549,San Francisco County


In [14]:
layoffs_by_county_2019 = cali_merge_2019.groupby(['County Names','Year']).agg('sum')
layoffs_2019 = layoffs_by_county_2019.merge(cali_population, how="left", on="County Names")
layoffs_2019.drop(columns=['Counties', 'Population_x'], inplace=True)
layoffs_2019.rename(columns={"Population_y":"Population"}, inplace=True)
layoffs_2019['Percentage Laid Off'] = (layoffs_2019['Employees']/layoffs_2019['Population']) * 100
layoffs_2019.sort_values(by='Percentage Laid Off', ascending=False)

,County Names,Employees,Population,Percentage Laid Off
1,Butte County,1877,219186,0.856350
12,Nevada County,521,99755,0.522280
5,Glenn County,143,28393,0.503645
24,Santa Clara County,8882,1927852,0.460720
19,San Francisco County,3808,881549,0.431967
0,Alameda County,4816,1671329,0.288154
22,San Mateo County,1869,766573,0.243812
34,Yolo County,479,220500,0.217234
7,Kern County,1805,900202,0.200511
14,Placer County,769,398329,0.193056


In [15]:
layoff_sum_2020 = layoffs['Employees'].sum()
layoff_sum_2020 

622600

In [16]:
layoff_sum_2019 = layoffs_2019['Employees'].sum()
layoff_sum_2019

66194

In [17]:
cali_merge_2019

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Fullerton,Orange County,204,Closure Permanent,2019,3175692,Orange County
1,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Buena Park,Orange County,45,Layoff Permanent,2019,3175692,Orange County
2,12/20/2019,12/20/2019,12/30/2019,"Event Solutions International, Inc. dba MotusO...",Long Beach,Los Angeles County,38,Layoff Permanent,2019,10039107,Los Angeles County
3,12/30/2019,12/31/2019,12/30/2019,Taylor Lane Organic Coffee,Petaluma,Sonoma County,6,Closure Permanent,2019,494336,Sonoma County
4,10/16/2019,11/01/2019,12/27/2019,Barneys New York,San Francisco,San Francisco County,100,Closure Permanent,2019,881549,San Francisco County
...,...,...,...,...,...,...,...,...,...,...,...
770,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",Ontario,San Bernardino County,16,Closure Permanent,2019,2180085,San Bernardino County
771,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",San Diego,San Diego County,84,Closure Permanent,2019,3338330,San Diego County
772,06/28/2019,08/30/2019,06/28/2019,"Coloredge, Inc.",Burbank,Los Angeles County,32,Closure Permanent,2019,10039107,Los Angeles County
773,06/28/2019,08/31/2019,06/28/2019,Verity Business Services,San Jose,Santa Clara County,5,Layoff Permanent,2019,1927852,Santa Clara County


In [18]:
cali_2019['Layoff/Closure'].unique()

array(['Closure Permanent', 'Layoff Permanent',
       'Layoff Unknown at this time', 'Closure Unknown at this time',
       'Layoff Temporary', 'Closure Temporary'], dtype=object)

In [19]:
cali_2019['Layoff/Closure'] = cali_2019['Layoff/Closure'].str.replace('at thistime', 'at this time')

<ipython-input-19-1bfdab73f880>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2019['Layoff/Closure'] = cali_2019['Layoff/Closure'].str.replace('at thistime', 'at this time')


In [20]:
cali_2019['Layoff/Closure'].unique()

array(['Closure Permanent', 'Layoff Permanent',
       'Layoff Unknown at this time', 'Closure Unknown at this time',
       'Layoff Temporary', 'Closure Temporary'], dtype=object)

In [21]:
cali_2020['Layoff/Closure'].unique()

array(['Closure Permanent', 'Layoff Permanent', 'Layoff Temporary',
       'Closure Temporary', 'Layoff Type Unknown', 'Closure Type Unknown',
       'Closure Unknown at this time', 'Layoff Unknown at this time'],
      dtype=object)

In [22]:
layoff_closures_2020 = cali_2020.groupby(['Layoff/Closure']).agg('sum')
layoff_closures_2020

,Employees
Layoff/Closure,
Closure Permanent,30692
Closure Temporary,72035
Closure Type Unknown,599
Closure Unknown at this time,3207
Layoff Permanent,107943
Layoff Temporary,383178
Layoff Type Unknown,5701
Layoff Unknown at this time,19245


In [23]:
cali_2020.groupby(['County','Layoff/Closure']).agg('sum')

Employees
County         Layoff/Closure                        
Alameda County Closure Permanent                 2297
               Closure Temporary                 1527
               Layoff Permanent                  4118
               Layoff Temporary                 24285
               Layoff Type Unknown                379
...                                               ...
Yolo County    Layoff Type Unknown                 20
               Layoff Unknown at this time        126
Yuba County    Closure Permanent                   80
               Layoff Permanent                    78
               Layoff Temporary                     1

[210 rows x 1 columns]

In [24]:
cali_2020.groupby(['County','Layoff/Closure']).agg('sum').reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Alameda County,Closure Permanent,2297
Alameda County,Closure Temporary,1527
Alameda County,Layoff Permanent,4118
Alameda County,Layoff Temporary,24285
Alameda County,Layoff Type Unknown,379
...,...,...
Yolo County,Layoff Type Unknown,20
Yolo County,Layoff Unknown at this time,126
Yuba County,Closure Permanent,80


In [25]:
layoff_closures_2019 = cali_2019.groupby(['Layoff/Closure']).agg('sum')
layoff_closures_2019

,Employees
Layoff/Closure,
Closure Permanent,28672
Closure Temporary,3272
Closure Unknown at this time,1700
Layoff Permanent,28438
Layoff Temporary,1775
Layoff Unknown at this time,2337


In [26]:
cali_2019.groupby(['County','Layoff/Closure']).agg('sum')

Employees
County         Layoff/Closure                         
Alameda County Closure Permanent                  2615
               Closure Unknown at this time        193
               Layoff Permanent                   1730
               Layoff Temporary                    191
               Layoff Unknown at this time          87
...                                                ...
Ventura County Layoff Unknown at this time          60
Yolo County    Closure Permanent                   443
               Layoff Permanent                     31
               Layoff Unknown at this time           5
Yuba County    Closure Permanent                    88

[97 rows x 1 columns]

In [27]:
cali_2019.groupby(['County','Layoff/Closure']).agg('sum').reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Alameda County,Closure Permanent,2615
Alameda County,Closure Unknown at this time,193
Alameda County,Layoff Permanent,1730
Alameda County,Layoff Temporary,191
Alameda County,Layoff Unknown at this time,87
...,...,...
Ventura County,Layoff Unknown at this time,60
Yolo County,Closure Permanent,443
Yolo County,Layoff Permanent,31


In [42]:
pd.options.display.max_rows = 999

In [53]:
cali_2020[cali_2020['County'] == 'Butte County']
# Lulu's Fashion Lounge - how to count the layoffs, 213 started off as temporary, 78 turned permanent
# Biggest = Associated Students of California StateUnivers with 660, but CSU has other layoffs, so do we attribute everything to CSU?

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
224,09/25/2020,08/19/2020,09/29/2020,"CSU, Chico",Chico,Butte County,446,Layoff Permanent,2020
651,04/23/2020,04/24/2020,09/01/2020,"Lulu's Fashion Lounge, LLC",Chico,Butte County,78,Layoff Permanent,2020
1997,04/21/2020,04/10/2020,05/27/2020,Associated Students of California StateUnivers...,Chico,Butte County,660,Layoff Temporary,2020
2536,03/23/2020,03/23/2020,05/14/2020,"Lulu's Fashion Lounge, LLC",Chico,Butte County,213,Layoff Temporary,2020
3100,04/08/2020,03/17/2020,04/29/2020,Restec Contractors Inc. at CSU Chico,Chico,Butte County,8,Layoff Temporary,2020
3205,04/17/2020,04/12/2020,04/28/2020,Dick's Sporting Goods Inc. at Chico Store#1089,Chico,Butte County,28,Layoff Temporary,2020
3770,04/02/2020,03/24/2020,04/20/2020,Pancakes 1702 INC,Chico,Butte County,45,Layoff Unknown at this time,2020
4173,03/26/2020,03/26/2020,04/15/2020,Cinemark USA Inc. dba Cinemark Chico,Chico,Butte County,46,Layoff Permanent,2020
4399,03/30/2020,03/16/2020,04/13/2020,"Hupp Draft Services, Inc.",Chico,Butte County,93,Layoff Temporary,2020
5615,12/18/2019,02/01/2020,12/19/2019,Adventist Health Feather River Palliative Care,Chico,Butte County,8,Closure Permanent,2020


In [59]:
cali_2020[cali_2020['County'] == 'Santa Cruz County']
#Fox Factory Inc looks like it has a double entry - the only thing different is the received date 
#(5/13 and 5/15 - everything else is exactly the same)

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
106,07/31/2020,09/01/2020,10/12/2020,"Mount Hermon Association, Inc.",Felton,Santa Cruz County,265,Layoff Permanent,2020
274,06/18/2020,06/26/2020,09/28/2020,1440 Multiversity,Scotts Valley,Santa Cruz County,102,Layoff Permanent,2020
367,06/01/2020,06/01/2020,09/23/2020,"Benchmark Conference Resorts of California, LL...",Santa Cruz,Santa Cruz County,125,Layoff Permanent,2020
636,04/09/2020,04/10/2020,09/02/2020,Watsonville Community Hospital,Watsonville,Santa Cruz County,7,Layoff Permanent,2020
638,04/14/2020,04/21/2020,09/02/2020,Watsonville Community Hospital,Watsonville,Santa Cruz County,23,Layoff Temporary,2020
884,08/06/2020,07/25/2020,08/07/2020,"BPR Properties Management, LLC dba BPR HOTELS,...",Santa Cruz,Santa Cruz County,82,Layoff Permanent,2020
1370,06/16/2020,08/21/2020,06/25/2020,Elyxir Distributing LLC,Watsonville,Santa Cruz County,113,Closure Permanent,2020
1503,04/02/2020,03/20/2020,06/16/2020,Central Coast YMCA,Watsonville,Santa Cruz County,41,Closure Temporary,2020
1585,05/27/2020,05/29/2020,06/09/2020,Movement for Life,Capitola,Santa Cruz County,3,Layoff Permanent,2020
1586,06/02/2020,05/18/2020,06/09/2020,"BPR Properties Management, LLC dba BPRHOTELS, ...",Santa Cruz,Santa Cruz County,68,Layoff Permanent,2020


In [45]:
cali_2020[cali_2020['County'] == 'Yolo County']

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
588,08/24/2020,08/21/2020,09/08/2020,Catalyst Family Inc,Madison,Yolo County,8,Layoff Type Unknown,2020
641,08/24/2020,08/21/2020,09/02/2020,Catalyst Family Inc.,Davis,Yolo County,12,Layoff Type Unknown,2020
794,08/07/2020,10/07/2020,08/17/2020,Raley's,West Sacramento,Yolo County,5,Layoff Temporary,2020
905,04/03/2020,04/13/2020,08/05/2020,Quest Diagnostics,Woodland,Yolo County,3,Layoff Temporary,2020
1013,07/28/2020,08/31/2020,07/28/2020,test,West Sacramento,Yolo County,52,Layoff Temporary,2020
1059,03/18/2020,03/20/2020,07/23/2020,Goodwill Industries of Sacramento Valley and N...,West Sacramento,Yolo County,21,Layoff Temporary,2020
1093,07/10/2020,07/10/2020,07/21/2020,Buckhorn Caf&eacute; Inc.,Winters,Yolo County,82,Layoff Permanent,2020
1094,07/10/2020,07/10/2020,07/21/2020,Buckhorn Cafe Inc.,Winters,Yolo County,29,Layoff Temporary,2020
1100,07/10/2020,07/25/2020,07/21/2020,"Philz Coffee, Inc.",Davis,Yolo County,6,Layoff Permanent,2020
1142,03/04/2020,03/17/2020,07/14/2020,"Seigo Enterprises, Inc. dba Zen Toro",Davis,Yolo County,12,Layoff Temporary,2020


In [58]:
pd.options.display.max_rows = 2000
cali_2020[cali_2020['County'] == 'Los Angeles County']

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
1,09/16/2020,04/27/2020,10/29/2020,Araca Merchandise LP.,San Fernando,Los Angeles County,89,Layoff Permanent,2020
2,10/23/2020,09/25/2020,10/29/2020,"STAR Sports Theatre Arts & Recreation, Inc.",Culver City,Los Angeles County,60,Layoff Permanent,2020
4,09/03/2020,05/08/2020,10/29/2020,Molson Coors Beverage Company,Baldwin Park,Los Angeles County,436,Closure Permanent,2020
8,08/20/2020,08/08/2020,10/28/2020,Aldo US Inc. Store 2072,Los Angeles,Los Angeles County,31,Layoff Temporary,2020
9,08/20/2020,08/08/2020,10/28/2020,"Aldo US Inc., Store 2081",Arcadia,Los Angeles County,7,Layoff Temporary,2020
16,10/09/2020,03/21/2020,10/27/2020,Equinox - Sports Club LA,Los Angeles,Los Angeles County,8,Layoff Permanent,2020
21,09/24/2020,07/12/2020,10/26/2020,Company 3 Method Inc. fka Deluxe Creative Serv...,Los Angeles,Los Angeles County,68,Layoff Temporary,2020
22,09/24/2020,07/12/2020,10/26/2020,Company 3 Method Inc. fka Deluxe Creative Serv...,Santa Monica,Los Angeles County,34,Layoff Temporary,2020
23,09/24/2020,07/26/2020,10/26/2020,Company 3 Method Inc. fka Deluxe Creative Serv...,Burbank,Los Angeles County,7,Layoff Temporary,2020
24,09/24/2020,07/31/2020,10/26/2020,Company 3 Method Inc. fka Deluxe Creative Serv...,Burbank,Los Angeles County,12,Layoff Permanent,2020


In [47]:
cali_2020[cali_2020['County'] == 'Orange County']

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
3,09/16/2020,04/25/2020,10/29/2020,Panasonic Avionics Corporation,Lake Forest,Orange County,582,Layoff Permanent,2020
14,10/23/2020,12/31/2020,10/27/2020,Pacific Life Insurance Company,Newport Beach,Orange County,87,Layoff Permanent,2020
48,10/15/2020,10/26/2020,10/22/2020,TGI Friday's Inc,Westminster,Orange County,44,Closure Permanent,2020
75,10/13/2020,10/30/2020,10/21/2020,Sheraton Operating Corporation DBA The Westin ...,Costa Mesa,Orange County,50,Layoff Permanent,2020
88,08/28/2020,10/30/2020,10/19/2020,The Boeing Company,Seal Beach,Orange County,10,Layoff Permanent,2020
147,09/30/2020,11/30/2020,10/05/2020,IHG Management LLC,Anaheim,Orange County,19,Layoff Permanent,2020
167,08/14/2020,08/14/2020,10/02/2020,Hyatt Regency Orange County,Garden Grove,Orange County,2,Layoff Permanent,2020
168,07/07/2020,07/03/2020,10/02/2020,Hyatt Corporation dba Hyatt Regency Orange County,Garden Grove,Orange County,52,Layoff Permanent,2020
169,06/30/2020,06/30/2020,10/02/2020,Hoag Memorial Hospital Presbyterian,Newport Beach,Orange County,218,Layoff Temporary,2020
170,09/29/2020,12/04/2020,10/02/2020,"Disney Destinations, LLC",Anaheim,Orange County,95,Layoff Permanent,2020


In [48]:
cali_2020[cali_2020['County'] == 'Fresno County']

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
95,10/09/2020,10/23/2020,10/14/2020,Adventist Health Reedley,Reedley,Fresno County,2,Layoff Permanent,2020
100,07/31/2020,07/31/2020,10/13/2020,Club One Casino,Fresno,Fresno County,126,Layoff Temporary,2020
538,09/01/2020,11/01/2020,09/10/2020,"Alorica, Inc.",Clovis,Fresno County,891,Closure Permanent,2020
591,05/15/2020,05/22/2020,09/04/2020,adventist Health Reedley,Reedley,Fresno County,1,Layoff Permanent,2020
592,05/15/2020,05/22/2020,09/04/2020,Adventist Health Reedley,Reedley,Fresno County,1,Layoff Temporary,2020
593,06/16/2020,06/26/2020,09/04/2020,Adventist Health Reedley,Reedley,Fresno County,6,Layoff Permanent,2020
1061,03/25/2020,03/20/2020,07/23/2020,Goodwill Industries Bulk Buying SJV,Fresno,Fresno County,24,Layoff Temporary,2020
1110,03/25/2020,03/20/2020,07/16/2020,Goodwill Industries - Fresno,Fresno,Fresno County,18,Layoff Temporary,2020
1150,04/03/2020,03/18/2020,07/13/2020,"Michael Cadillac, Inc. dba Michael Chevrolet, ...",Fresno,Fresno County,278,Layoff Temporary,2020
1326,04/03/2020,03/17/2020,06/26/2020,Club One Casino,Fresno,Fresno County,220,Closure Temporary,2020


In [49]:
cali_2020[cali_2020['County'] == 'Humboldt County']

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
1861,04/13/2020,03/17/2020,06/02/2020,Trumpet Behavioral Health - Arcata,Arcata,Humboldt County,43,Layoff Temporary,2020
1988,04/02/2020,04/02/2020,05/28/2020,"Kokatat, Inc",Arcata,Humboldt County,161,Layoff Temporary,2020
4299,03/30/2020,04/01/2020,04/14/2020,"Renson Hotels, Inc. dba Holiday Inn Express &S...",Mckinleyville,Humboldt County,23,Layoff Temporary,2020


In [50]:
layoffs_2020_and_2019 = pd.merge(layoffs, layoffs_2019, how="left", left_on="County Names", right_on="County Names")
layoffs_2020_and_2019.rename(columns={"Employees_x":"Employees_2020", "Population_x":"Population_2020", "Percentage Laid Off_x":"Percentage Laid Off_2020", "Employees_y":"Employees_2019","Percentage Laid Off_y":"Percentage Laid Off_2019"}, inplace=True)
layoffs_2020_and_2019.drop(columns='Population_y', inplace=True)
layoffs_2020_and_2019.sort_values(by='Percentage Laid Off_2020', ascending=False)

,County Names,Employees_2020,Population_2020,Percentage Laid Off_2020,Employees_2019,Percentage Laid Off_2019
18,Mariposa County,2262,17203,13.148869,<NA>,NaN
33,San Francisco County,56652,881549,6.426415,3808,0.431967
21,Mono County,879,14444,6.085572,<NA>,NaN
12,Inyo County,539,18039,2.987971,<NA>,NaN
32,San Diego County,75458,3338330,2.260352,4072,0.121977
36,San Mateo County,16987,766573,2.215966,1869,0.243812
23,Napa County,2975,137744,2.159804,110,0.079858
25,Orange County,64764,3175692,2.039367,4427,0.139403
0,Alameda County,32756,1671329,1.959877,4816,0.288154
15,Los Angeles County,191307,10039107,1.905618,15219,0.151597


In [51]:
# cali_2019[cali_2019['County'] == "Mariposa County"]
# just a check

## Next Steps

- [x] Standardize the county names
- [x] calculate percentage of eviction per county
- [x] write up findings for Evictions-CA google doc
- [ ] Get city names and county names to match up the cities with 'No County Name' in County column (later for information prior to 2020 - which adds context)
- [ ] combine population information with fips codes to get the population for each city